## PromptTemplate

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "CH02-Prompt"
os.environ["LANGCHAIN_PROJECT"] = project_name

LLM 객체를 정의한다

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 방법1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성

- 치환될 변수를 `{ 변수 }`로 묶어서 템플릿을 정의한다.

In [3]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

`country` 변수에 값을 넣어서 문장을 생성할 수 있다.

In [4]:
# prompt 생성. format 메소드를 이용하여 변수에 값을 넣어줌
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

In [8]:
# country 변수에 입력된 값이 자동으로 치환되어 수행됨
chain.invoke({"country":"대한민국"}).content

'대한민국의 수도는 서울입니다.'

### 방법2. PromptTemplate 객체 생성과 동시에 prompt 생성

추가 유효성 검사를 위해 `input_variables`를 명시적으로 지정하세요.

이러변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치하경우 예외를 발생시킨다.

In [11]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"]
)

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [12]:
# prompt 생성
prompt.format(country="대한민국")

'대한민국의 수도는 어디인가요?'

In [13]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각  어디인가요?"

# PrmoptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국", # dictionary 형태로 partial_variables를 전달
    },
)

prompt

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각  어디인가요?')

In [14]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각  어디인가요?'

In [15]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country1', 'country2'], input_types={}, partial_variables={}, template='{country1}과 {country2}의 수도는 각각  어디인가요?')

In [16]:
prompt_partial = prompt.partial(country2="캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], input_types={}, partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각  어디인가요?')

In [17]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각  어디인가요?'

In [18]:
chain = prompt_partial | llm

In [19]:
chain.invoke("대한민국").content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [20]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이고, 호주의 수도는 캔버라입니다.'

### `partial_variables`: 부분 변수 채움

`partial`을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것이다. 이 사용 사례는 **항상 공통된 방식으로 가져오고 싶은 변수**가 있는 경우이다. 

대표적인 예가 **날씨나 시간** 이다.

항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 본다. 프롬프트에 하드 코딩할 수도 없고,다른 입력 변수와 함께 전달하는 것도 번거롭다. 이 경우 항상 현재 **날짜를 반환하는 함수**를 사용하여 프롬프트를  부분적으로 변경할 수 있으면 매우 편리하다. 

다음의 코드는 오늘 날짜를 구하는 파이썬 코드이다.

In [21]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'October 05'

In [22]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [23]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘의 생일인 유명인 {n}명을 나열에 주세요. 생년월일을 표기해주세요.",
    input_variables={"n"},
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    }
)
prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x117e59800>}, template='오늘의 날짜는 {today} 입니다. 오늘의 생일인 유명인 {n}명을 나열에 주세요. 생년월일을 표기해주세요.')

In [24]:
# prompt 생성
prompt.format(n=3)

'오늘의 날짜는 October 05 입니다. 오늘의 생일인 유명인 3명을 나열에 주세요. 생년월일을 표기해주세요.'

In [ ]:
# chain 생성
chain = prompt | ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [31]:
# chain을 실행 후 결과를 확인한다
print(chain.invoke(3).content)

October 5일에 태어난 유명인 중 세 명을 소개하겠습니다.

1. 케이트 윈슬렛 (Kate Winslet) - 1975년 10월 5일
2. 가이 피어스 (Guy Pearce) - 1967년 10월 5일
3. 닐 드그래스 타이슨 (Neil deGrasse Tyson) - 1958년 10월 5일

이들은 각각 영화, TV, 과학 분야에서 잘 알려진 인물들입니다.


In [32]:
# chain을 실행 후 결과를 확인한다
print(chain.invoke({"today": "Jan 02", "n": 3}).content)

1월 2일에 태어난 유명인 중 세 명을 소개하겠습니다.

1. **아이작 아시모프 (Isaac Asimov)** - 1920년 1월 2일 출생. 러시아 태생의 미국 작가이자 생화학자이며, 과학 소설과 과학 서적을 다수 집필한 것으로 유명합니다.

2. **큐반 굿잉 주니어 (Cuba Gooding Jr.)** - 1968년 1월 2일 출생. 미국의 배우로, 영화 "제리 맥과이어"에서의 연기로 아카데미 남우조연상을 수상했습니다.

3. **케이트 보스워스 (Kate Bosworth)** - 1983년 1월 2일 출생. 미국의 배우로, 영화 "블루 크러쉬"와 "슈퍼맨 리턴즈" 등에 출연했습니다.

이 외에도 여러 유명인들이 1월 2일에 태어났습니다.


## 파일로부터 template 읽어오기

In [33]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

Window 사용자 중 이전의 코드가 오류가 나는 경우 아래의 코드를 실행하세요(인코딩 설정)

In [ ]:
# from langchain_teddynote_prompts import load_prompt

# # Windows 사용자 only: 인코딩을 CP949로 설정
# prompt = load_prompt("prompts/fruit_color.yaml", encoding="cp949")

In [34]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [36]:
prompt2 = load_prompt("prompts/capital.yaml")
prompt2

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n')

In [37]:
print(prompt2.format(country="대한민국"))

대한민국의 수도에 대해서 알려주세요.
수도의 특징을 다음의 양식에 맞게 정리해 주세요.
300자 내외로 작성해 주세요.
한글로 작성해 주세요.
----
[양식]
1. 면적
2. 인구
3. 역사적 장소
4. 특산품

#Answer:



In [41]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt2 | ChatOpenAI(model="gpt-4o", temperature=0) | StrOutputParser()
chain

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11882e790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11880de90>, root_client=<openai.OpenAI object at 0x11882ec10>, root_async_client=<openai.AsyncOpenAI object at 0x11882df50>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [ ]:
answer = chain.stream({"country": "대한민"})
for 

## ChatPromptTemplate

`ChatPromptTemplate` 은 대화목록을 프롬프트로 주입하고자 할 때 활용할 수 있습니다.

메시지는 튜플(tuple) 형식으로 구성하며, (`role`, `message`) 로 구성하여 리스트로 생성할 수 있습니다.

**role**
- `"system"`: 시스템 설정 메시지 입니다. 주로 전역설정과 관련된 프롬프트입니다.
- `"human"` : 사용자 입력 메시지 입니다.
- `"ai"`: AI 의 답변 메시지입니다.

In [43]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [44]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [51]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}")
    ]
)

# 챗 message를 생성
messages = chat_template.format_messages(
    name="앤디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 앤디 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 이름은 무엇입니까?', additional_kwargs={}, response_metadata={})]

생성메시지를  지바로 주입하여 결과를 받을 수 있다

In [52]:
llm.invoke(messages).content

'제 이름은 앤디입니다. 어떻게 도와드릴까요?'

이번에는 체인을 생성해 보겠습니다

In [48]:
chain = chat_template | llm
chain

ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='반가워요!'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='안녕하세요! 무엇을 도와드릴까요?'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], input_types={}, partial_variables={}, template='{user_input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11743c2d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117cc7750>, root_client=<openai.O

In [49]:
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Teddy입니다. 어떻게 도와드릴까요?'

## MessagePlaceholer

또한 LangChain을 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는  `MessagePlaceholder` 를 제공한다.

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있다. 

In [54]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다.."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

`conversation` 대화목록을 나중에 추가하고자 할 때 `MessagePlaceholder`를 사용할 수 있다.

In [55]:
chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 앤디입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁드립니다.")
    ]
)

'Human: 안녕하세요! 저는 오늘 새로 입사한 앤디입니다. 만나서 반갑습니다.\nAI: 반가워요! 앞으로 잘 부탁드립니다.\nHuman: 지금까지의 대화를 5 단어로 요약합니다..'

In [58]:
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [59]:
# chain 생성
chain = chat_prompt | ChatOpenAI(temperature=0.1, model="gpt-4.1-mini")
chain

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [60]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 앤디입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁드립니다."),
        ]
    }
)

AIMessage(content='새로운 입사자 앤디 인사', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 61, 'total_tokens': 71, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_95d112f245', 'id': 'chatcmpl-CN8QYczqSDOgxgQixj20wgAFQKbsx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd345171-d3de-4cfa-b123-0e07f70e59ab-0', usage_metadata={'input_tokens': 61, 'output_tokens': 10, 'total_tokens': 71, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})